## Bilansihälvete ja kulu arvutamine

- loeme failist `data/balance_input.json`
- arvutame tarbimise ja tootmise bilansihälbed
- arvutame hälbekulu iga perioodi kohta
- koondame tulemused

Impordid ja teed

In [66]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

In [80]:
project_root = Path("..").resolve()
data_dir = project_root / "data"

balance_input_path = data_dir / "balance_input.json"
price_path = data_dir / "imbalance_prices_2025-12-10_2025-12-12.json"

Andmete laadimine

In [81]:
df = pd.read_json(balance_input_path)
df.head(), df.columns

(            timestamp  consumption_mwh  forecast_consumption_mwh  \
 0 2025-12-09 22:00:00           923.18                     823.4   
 1 2025-12-09 22:15:00           929.09                     817.6   
 2 2025-12-09 22:30:00           901.28                     807.9   
 3 2025-12-09 22:45:00           910.34                     823.9   
 4 2025-12-09 23:00:00           860.79                     760.3   
 
    production_mwh  forecast_production_mwh       date  hour  minute  \
 0          440.98                    401.0 2025-12-09    22       0   
 1          433.93                    373.7 2025-12-09    22      15   
 2          435.96                    376.0 2025-12-09    22      30   
 3          427.05                    377.2 2025-12-09    22      45   
 4          439.33                    349.0 2025-12-09    23       0   
 
    day_of_week  is_weekend  
 0            1       False  
 1            1       False  
 2            1       False  
 3            1       False  


Bilansi hinna JSON faili import

In [82]:
with open(price_path, "r", encoding="utf-8") as f:
    price_json = json.load(f)

price_json.keys(), price_json["data"].keys()

(dict_keys(['error', 'message', 'data']),
 dict_keys(['id', 'export_id', 'measurement_unit', 'resolution', 'precision', 'precision_mode', 'default_display', 'title', 'description', 'timezone', 'local_timezone', 'daylight_saving_transitions', 'creation_time', 'column_group_levels', 'columns', 'timeseries']))

Hind DataFrame'i

In [83]:
ts = price_json["data"]["timeseries"]
df_price = pd.DataFrame(ts)

df_price.head()

,from,to,values
0,2025-12-09T22:00:00+00:00,2025-12-09T22:15:00+00:00,"[None, 30.63, None, None, None, 4.896]"
1,2025-12-09T22:15:00+00:00,2025-12-09T22:30:00+00:00,"[None, 72.52, None, None, None, -27.172]"
2,2025-12-09T22:30:00+00:00,2025-12-09T22:45:00+00:00,"[None, 21.76, None, None, None, -16.457]"
3,2025-12-09T22:45:00+00:00,2025-12-09T23:00:00+00:00,"[None, 28.82, None, None, None, -14.275]"
4,2025-12-09T23:00:00+00:00,2025-12-09T23:15:00+00:00,"[None, 65.49, None, None, None, -10.244]"


Eesti hind

In [84]:
df_price["from_dt_utc"] = pd.to_datetime(df_price["from"], utc=True)
df_price["ts_key"] = df_price["from_dt_utc"].astype("int64") // 10**9

def get_estonia_price(values):
    # Estonia Preliminary = index 1
    if not isinstance(values, list) or len(values) <= 1:
        return None
    return values[1]

df_price["imbalance_price_eur_mwh"] = df_price["values"].apply(get_estonia_price)

df_price[["from_dt_utc", "ts_key", "imbalance_price_eur_mwh"]].head()

,from_dt_utc,ts_key,imbalance_price_eur_mwh
0,2025-12-09 22:00:00+00:00,1765317600,30.63
1,2025-12-09 22:15:00+00:00,1765318500,72.52
2,2025-12-09 22:30:00+00:00,1765319400,21.76
3,2025-12-09 22:45:00+00:00,1765320300,28.82
4,2025-12-09 23:00:00+00:00,1765321200,65.49


Ühendame hinnad bilansitabeliga

In [85]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["ts_key"] = df["timestamp"].astype("int64") // 10**9

for col in list(df.columns):
    if "imbalance_price_eur_mwh" in col:
        df = df.drop(columns=[col])

df = pd.merge(
    df,
    df_price[["ts_key", "imbalance_price_eur_mwh"]],
    on="ts_key",
    how="left",
)

df = df.drop(columns=["ts_key"])

df[["timestamp", "consumption_mwh", "imbalance_price_eur_mwh"]].head()


,timestamp,consumption_mwh,imbalance_price_eur_mwh
0,2025-12-09 22:00:00,923.18,30.63
1,2025-12-09 22:15:00,929.09,72.52
2,2025-12-09 22:30:00,901.28,21.76
3,2025-12-09 22:45:00,910.34,28.82
4,2025-12-09 23:00:00,860.79,65.49


Bilansihälbed (MWh)
   - tarbimise hälbe = tegelik tarbimine - prognoositud tarbimine
   - tootmise hälbe = tegelik tootmine - prognoositud tootmine

In [97]:
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Kui veerus on Nan, asendame 0-ga, et arvutada
df["production_mwh"] = df["production_mwh"].fillna(0)
df["forecast_production_mwh"] = df["forecast_production_mwh"].fillna(0)

# Tarbimise bilansihälve
df["consumption_imbalance_mwh"] = df["consumption_mwh"] - df["forecast_consumption_mwh"]

# Tootmise bilansihälve
df["production_imbalance_mwh"] = df["production_mwh"] - df["forecast_production_mwh"]

# Netohälve (tootmine - tarbimine)
df["net_imbalance_mwh"] = df["production_imbalance_mwh"] - df["consumption_imbalance_mwh"]
df[[
    "timestamp",
    "consumption_mwh",
    "forecast_consumption_mwh",
    "consumption_imbalance_mwh",
    "production_mwh",
    "forecast_production_mwh",
    "production_imbalance_mwh",
    "net_imbalance_mwh"
]].head()

,timestamp,consumption_mwh,forecast_consumption_mwh,consumption_imbalance_mwh,production_mwh,forecast_production_mwh,production_imbalance_mwh,net_imbalance_mwh
0,2025-12-09 22:00:00,923.18,823.4,99.78,440.98,401.0,39.98,-59.80
1,2025-12-09 22:15:00,929.09,817.6,111.49,433.93,373.7,60.23,-51.26
2,2025-12-09 22:30:00,901.28,807.9,93.38,435.96,376.0,59.96,-33.42
3,2025-12-09 22:45:00,910.34,823.9,86.44,427.05,377.2,49.85,-36.59
4,2025-12-09 23:00:00,860.79,760.3,100.49,439.33,349.0,90.33,-10.16


Kulude arvutamine

In [98]:
# Tarbimise hälbekulu
df["consumption_imbalance_cost_eur"] = (
    df["consumption_imbalance_mwh"] * df["imbalance_price_eur_mwh"]
)

# Tootmise hälbekulu
df["production_imbalance_cost_eur"] = (
    df["production_imbalance_mwh"] * df["imbalance_price_eur_mwh"]
)

# Netohälbe kulu
df["imbalance_cost_eur"] = (
    df["production_imbalance_cost_eur"] + df["consumption_imbalance_cost_eur"]
)

df[[
    "timestamp",
    "imbalance_price_eur_mwh",
    "consumption_imbalance_mwh",
    "production_imbalance_mwh",
    "net_imbalance_mwh",
    "imbalance_cost_eur"
]].head()


,timestamp,imbalance_price_eur_mwh,consumption_imbalance_mwh,production_imbalance_mwh,net_imbalance_mwh,imbalance_cost_eur
0,2025-12-09 22:00:00,30.63,99.78,39.98,-59.80,4280.8488
1,2025-12-09 22:15:00,72.52,111.49,60.23,-51.26,12453.1344
2,2025-12-09 22:30:00,21.76,93.38,59.96,-33.42,3336.6784
3,2025-12-09 22:45:00,28.82,86.44,49.85,-36.59,3927.8778
4,2025-12-09 23:00:00,65.49,100.49,90.33,-10.16,12496.8018


Päevatabel

In [101]:
settlement_daily = (
    df.groupby("date")
    .agg(
        total_consumption_mwh=("consumption_mwh", "sum"),
        total_forecast_consumption_mwh=("forecast_consumption_mwh", "sum"),
        total_production_mwh=("production_mwh", "sum"),
        total_forecast_production_mwh=("forecast_production_mwh", "sum"),
        consumption_imbalance_mwh=("consumption_imbalance_mwh", "sum"),
        production_imbalance_mwh=("production_imbalance_mwh", "sum"),
        net_imbalance_mwh=("net_imbalance_mwh", "sum"),
        consumption_imbalance_cost_eur=("consumption_imbalance_cost_eur", "sum"),
        production_imbalance_cost_eur=("production_imbalance_cost_eur", "sum"),
        imbalance_cost_eur=("imbalance_cost_eur", "sum"),
        avg_imbalance_price_eur_mwh=("imbalance_price_eur_mwh", "mean"),
    )
    .reset_index()
)

settlement_daily.head()


,date,total_consumption_mwh,total_forecast_consumption_mwh,total_production_mwh,total_forecast_production_mwh,consumption_imbalance_mwh,production_imbalance_mwh,net_imbalance_mwh,consumption_imbalance_cost_eur,production_imbalance_cost_eur,imbalance_cost_eur,avg_imbalance_price_eur_mwh
0,2025-12-09,7112.08,6287.2,3449.19,2933.5,824.88,515.69,-309.19,16834.1395,8886.2657,25720.4052,21.225000
1,2025-12-10,99845.57,93893.0,54912.35,49391.5,5952.57,5520.85,-431.72,356376.8951,303813.6318,660190.5269,61.955417
2,2025-12-11,84164.63,86908.0,42003.84,40169.0,4252.13,1834.84,-165.19,107697.2731,35387.5745,143084.8476,16.342368


Salvestame JSON-i

In [100]:
balance_with_imbalance_path = data_dir / "balance_with_imbalance.json"
settlement_daily_path = data_dir / "settlement_daily.json"

df.to_json(balance_with_imbalance_path, orient="records", date_format="iso")
settlement_daily.to_json(settlement_daily_path, orient="records", date_format="iso")

balance_with_imbalance_path, settlement_daily_path


(WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/balance_with_imbalance.json'),
 WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/settlement_daily.json'))

- kasutatud Eleringi esialgseid bilansihindu
